First thing we want to do is create a staging table. This is the one we will work in and clean the data. We want a table with the raw data in case something happens

In [ ]:
SELECT * 
FROM world_layoffs.layoffs;
CREATE TABLE world_layoffs.layoffs_staging 
LIKE world_layoffs.layoffs;

INSERT layoffs_staging 
SELECT * FROM world_layoffs.layoffs;

Steps for the cleaning process:

Step 1. check for duplicates and remove any
Step 2. standardize data and fix errors
Step 3. Look at null values 
Step 4. remove any columns and rows that are not necessary 

Step 1. Remove Duplicates
First we are going to check for duplicates 

In [ ]:
SELECT *
FROM world_layoffs.layoffs_staging
;

SELECT company, industry, total_laid_off,`date`,
		ROW_NUMBER() OVER (
			PARTITION BY company, industry, total_laid_off,`date`) AS row_num
	FROM 
		world_layoffs.layoffs_staging;

In [ ]:
SELECT *
FROM (
	SELECT company, industry, total_laid_off,`date`,
		ROW_NUMBER() OVER (
			PARTITION BY company, industry, total_laid_off,`date`
			) AS row_num
	FROM 
		world_layoffs.layoffs_staging
) duplicates
WHERE 
	row_num > 1;

Lets look at a company to confirm, we will look at the company called Oda

In [ ]:
SELECT *
FROM world_layoffs.layoffs_staging
WHERE company = 'Oda'

After running the queries it looks like these are legitimate entries. We need to look at every single row to be accurate. This is a manual process but can work for a data set of this size

Running the query to list down the real duplicates

In [ ]:
SELECT *
FROM (
	SELECT company, location, industry, total_laid_off,percentage_laid_off,`date`, stage, country, funds_raised_millions,
		ROW_NUMBER() OVER (
			PARTITION BY company, location, industry, total_laid_off,percentage_laid_off,`date`, stage, country, funds_raised_millions
			) AS row_num
	FROM 
		world_layoffs.layoffs_staging
) duplicates
WHERE 
	row_num > 1;

By running the above query we get the duplicate items. These are the ones we want to delete where the row number is > 1 or 2 or greater essentially.

We will now delete the duplicates using a CTE

In [ ]:
WITH DELETE_CTE AS 
(
SELECT *
FROM (
	SELECT company, location, industry, total_laid_off,percentage_laid_off,`date`, stage, country, funds_raised_millions,
		ROW_NUMBER() OVER (
			PARTITION BY company, location, industry, total_laid_off,percentage_laid_off,`date`, stage, country, funds_raised_millions
			) AS row_num
	FROM 
		world_layoffs.layoffs_staging
) duplicates
WHERE 
	row_num > 1
)
DELETE
FROM DELETE_CTE
;

In [ ]:
WITH DELETE_CTE AS (
	SELECT company, location, industry, total_laid_off, percentage_laid_off, `date`, stage, country, funds_raised_millions, 
    ROW_NUMBER() OVER (PARTITION BY company, location, industry, total_laid_off, percentage_laid_off, `date`, stage, country, funds_raised_millions) AS row_num
	FROM world_layoffs.layoffs_staging
)
DELETE FROM world_layoffs.layoffs_staging
WHERE (company, location, industry, total_laid_off, percentage_laid_off, `date`, stage, country, funds_raised_millions, row_num) IN (
	SELECT company, location, industry, total_laid_off, percentage_laid_off, `date`, stage, country, funds_raised_millions, row_num
	FROM DELETE_CTE
) AND row_num > 1;

One solution which I can think of is to create a new column and add those row numbers in. Then delete where row numbers are over 2, then delete that column

In [ ]:
ALTER TABLE world_layoffs.layoffs_staging ADD row_num INT;

In [ ]:
SELECT *
FROM world_layoffs.layoffs_staging
;

In [ ]:
CREATE TABLE `world_layoffs`.`layoffs_staging2` (
`company` text,
`location`text,
`industry`text,
`total_laid_off` INT,
`percentage_laid_off` text,
`date` text,
`stage`text,
`country` text,
`funds_raised_millions` int,
row_num INT
);

In [ ]:
INSERT INTO `world_layoffs`.`layoffs_staging2`
(`company`,
`location`,
`industry`,
`total_laid_off`,
`percentage_laid_off`,
`date`,
`stage`,
`country`,
`funds_raised_millions`,
`row_num`)
SELECT `company`,
`location`,
`industry`,
`total_laid_off`,
`percentage_laid_off`,
`date`,
`stage`,
`country`,
`funds_raised_millions`,
		ROW_NUMBER() OVER (
			PARTITION BY company, location, industry, total_laid_off,percentage_laid_off,`date`, stage, country, funds_raised_millions
			) AS row_num
	FROM 
		world_layoffs.layoffs_staging;

Now that we have this we can delete rows were row_num is greater than 2

In [ ]:
DELETE FROM world_layoffs.layoffs_staging2
WHERE row_num >= 2;

Step 2 - Standardize Data

In [ ]:
SELECT * 
FROM world_layoffs.layoffs_staging2;

In the indsutry columns, there seem to be some null values. Will take a closer look ay them now.

In [ ]:
SELECT DISTINCT industry
FROM world_layoffs.layoffs_staging2
ORDER BY industry;

In [ ]:
SELECT *
FROM world_layoffs.layoffs_staging2
WHERE industry IS NULL 
OR industry = ''
ORDER BY industry;

Lets look like lookalike companies 

In [ ]:
SELECT *
FROM world_layoffs.layoffs_staging2
WHERE company LIKE 'Bally%';

There seems to be nothing wrong here.

In [ ]:
SELECT *
FROM world_layoffs.layoffs_staging2
WHERE company LIKE 'airbnb%';

It looks like airbnb is a travel company, but this one just isn't populated.
I'm sure it's the same for the others. What we can do is write a query that if there is another row with the same company name, it will update it to the non-null industry values
This makes it easy so if there were thousands we wouldn't have to manually check them all

Before doing so we can set the blanks to nulls since they are easier to work with.

In [ ]:
UPDATE world_layoffs.layoffs_staging2
SET industry = NULL
WHERE industry = '';

Lets check if all are null now or not.

In [ ]:
SELECT *
FROM world_layoffs.layoffs_staging2
WHERE industry IS NULL 
OR industry = ''
ORDER BY industry;

We will now populate nulls

In [ ]:
UPDATE layoffs_staging2 t1
JOIN layoffs_staging2 t2
ON t1.company = t2.company
SET t1.industry = t2.industry
WHERE t1.industry IS NULL
AND t2.industry IS NOT NULL;

If we check it looks like Bally's was the only one without a populated row to populate this null value.

In [ ]:
SELECT *
FROM world_layoffs.layoffs_staging2
WHERE industry IS NULL 
OR industry = ''
ORDER BY industry;

Crypto has multiple different variations. We need to standardize that.

In [ ]:
SELECT DISTINCT industry
FROM world_layoffs.layoffs_staging2
ORDER BY industry;

In [ ]:
UPDATE layoffs_staging2
SET industry = 'Crypto'
WHERE industry IN ('Crypto Currency', 'CryptoCurrency');

Looking at the industry column final time to check.

In [ ]:
SELECT DISTINCT industry
FROM world_layoffs.layoffs_staging2
ORDER BY industry;

Looking at the complete table

In [ ]:
SELECT *
FROM world_layoffs.layoffs_staging2;

Eeverything looks good except we have "United States" and some "United States." with a period at the end. We will now standardize this.

In [ ]:
SELECT DISTINCT country
FROM world_layoffs.layoffs_staging2
ORDER BY country;

In [ ]:
UPDATE layoffs_staging2
SET country = TRIM(TRAILING '.' FROM country);


We need to fix the date columns using str to date to update this field.

In [ ]:
UPDATE layoffs_staging2
SET `date` = STR_TO_DATE(`date`, '%m/%d/%Y');

Now we can convert the data type

In [ ]:
ALTER TABLE layoffs_staging2
MODIFY COLUMN `date` DATE;

Step 3 - Remove null values
The null values in total_laid_off, percentage_laid_off, and funds_raised_millions all look normal. I don't think I want to change that
I like having them null because it makes it easier for calculations during the EDA phase
So there isn't anything I want to change with the null values

Step 4 Remove columns and rows that are not necessary 

In [ ]:
DELETE FROM world_layoffs.layoffs_staging2
WHERE total_laid_off IS NULL
AND percentage_laid_off IS NULL;

In [ ]:
ALTER TABLE layoffs_staging2
DROP COLUMN row_num;

Our table is now cleaned and ready for data analysis